In [1]:
# mongodb functions
from pymongo import MongoClient
# remember to start te mongodb server: sudo service mongod start

# scrapping functions
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# sql functions
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

# other
import pickle

# get id and url info from mongodb

In [2]:
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient["kick"] #pyramids is the database
mycol = mydb["k1912"] #invoice is the collection

In [ ]:
# get the urls
id_urls = []

for proposal in mycol.find({"data.state": { "$in": ["successful", "failed"]}},
                           {"data.urls.web": 1, "data.blurb" : 1, "data.state" : 1, "data.id" :1}):
    urls            = proposal['data']['urls']['web']['project']
    blurbs_finished = proposal['data']['blurb']
    blurbs_state    = proposal['data']['state']
    ids             = proposal['data']['id']
    id_urls = [urls, blurbs_finished, blurbs_state, ids]

## create browser

In [3]:
options = Options()
options.add_argument('-headless')
browser = webdriver.Firefox(options = options)

## connect to postgre database

In [2]:
dbname = 'birth_db'
username = 'shengsql'
pswd = '1234'

# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)
cursor = con.cursor()

In [3]:
# example query:
sql_query = """SELECT project_id FROM stories;"""

birth_data_from_sql = pd.read_sql_query(sql_query,con)
birth_data_from_sql.tail(3)
#2204

,project_id
2202,1.660540e+09
2203,1.781537e+09
2204,9.153386e+08


## scrapping the kickstarter website

In [ ]:
inser_300 = "INSERT INTO stories(project_id, url, story) VALUES(%s, %s, %s)"

for i in range(1853, 2500):
    b = browser
    p_id = id_urls[3][i]
    p_url = id_urls[0][i]
    b.get(p_url)
    try:
        element = WebDriverWait(browser, 100).until(
            EC.presence_of_element_located((By.CLASS_NAME, "rte__content"))
        ) # this part ensures that the next command wait until the element loads 
    finally:
        s = b.find_element_by_class_name("rte__content").text.replace('\n','').lower()
        record = [p_id, p_url, s]
        cursor.execute(inser_300, record)
        con.commit() 

## scrape more failed project
b.c. the first part of the url list had more successful projects than failed, not representing the population

In [17]:
# query all project information, joining project metadata with storis info:
sql_query = """SELECT projects.project_id, projects.url, state
    FROM projects LEFT JOIN stories 
    ON (projects.project_id = stories.project_id) 
    WHERE stories.project_id IS NULL 
    AND state = 'failed';"""

failed_project_url = pd.read_sql_query(sql_query,con)

In [24]:
failed_project_url.url[0]

'https://www.kickstarter.com/projects/378101615/nowa-ukraine?ref=discovery_category_newest'

In [ ]:
#for i in range(1000)

inser_300 = "INSERT INTO stories(project_id, url, story) VALUES(%s, %s, %s)"

for i in range(147, 700):
    b = browser
    p_id = str(failed_project_url.project_id[i])
    p_url = failed_project_url.url[i]
    b.get(p_url)
    try:
        element = WebDriverWait(browser, 100).until(
            EC.presence_of_element_located((By.CLASS_NAME, "rte__content"))
        ) # this part ensures that the next command wait until the element loads 
    finally:
        s = b.find_element_by_class_name("rte__content").text.replace('\n','').lower()
        record = [p_id, p_url, s]
        cursor.execute(inser_300, record)
        con.commit() 

# scrape an example for the app

In [5]:
url_app_example = "https://www.kickstarter.com/projects/blackdiplomats/black-diplomats-decolonize-the-global-affairs-conversation?ref=discovery&term=Black%20Diplomats%20-%20Decolonize%20the%20global%20affairs%20conversation"
browser.get(url_app_example)
try:
  element = WebDriverWait(browser, 100).until(
    EC.presence_of_element_located((By.CLASS_NAME, "rte__content")))
finally:
  example_story = browser.find_element_by_class_name("rte__content").text.replace('\n','').lower()

In [7]:
with open("pickle/app_example.pkl", "wb") as f:
    pickle.dump(example_story, f)